In [ ]:
import ray
ray.shutdown()

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_ws_api as pws
import polygon_df as pdf
import polygon_ds as pds
import polygon_s3 as ps3

import bar_samples as bs
import bar_labels as bl
import bar_meta as bm
import filters as ft

import flow_backfill as fb

# import market_cluster as mc

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'polygon_backfill'

In [ ]:
symbol='GLD'
# symbol = 'market'
# date = '2020-01-02'
# start_date = '2020-01-01'
start_date = '2020-10-01'
end_date = '2020-10-10'
# symbols=['SPY', 'QQQ']
tick_type='trades'

thresh = {
    # max thresholds
    'renko_reveral_multiple': 2,
    'max_duration_sec': 60*60,
    # min thresholds
    'min_duration_sec': 15,
    'min_tick_count': 30,
}

horizon_mins=30

In [ ]:
df = ps3.load_ticks(sym, '2020-11-05')

In [ ]:
df = ps3.load_ticks(symbol, '2020-11-05')
sdf = df.loc[df.irregular==False]
sdf2 = ft.jma_filter_df(sdf, 'price', length=5, power=0.5)

In [ ]:
(abs(sdf2.price - sdf2.price_jma) / sdf2.price_jma).describe(percentiles=[.99,.999,.9995,.9999])

In [ ]:
fb.run_backfill(symbols=['GLD'], tick_type='trades', start_date='2020-10-01')
# fb.run_backfill(symbols=['market'], tick_type='daily', start_date='2020-10-01')

In [ ]:
import ray
ray.init(ignore_reinit_error=True, num_cpus=4, dashboard_port=8080)

daily_bar_stats_df, labeled_bar_dates, stacked_bars_df = bm.bars_workflow_ray( 
    symbol, 
    start_date, 
    end_date, 
    thresh,
    imbalance_thresh=0.9
)

In [ ]:
import pickle 
fio = open('bars_workflow_ray-new.pickle', 'wb') 
pickle.dump((daily_bar_stats_df, labeled_bar_dates, stacked_bars_df), fio)

In [ ]:
data = (daily_bar_stats_df, labeled_bar_dates, stacked_bars_df)
file = open('test.pickle', mode='wb')
pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("test.pickle", "rb") as input_file:
    e = pickle.load(input_file)

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])